In [2]:
from itertools import zip_longest

import pandas as pd

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', None)

In [4]:
df = pd.read_csv("../dataset_checking/data/new_isaiah_dataset.csv").fillna("")
### Uncomment to exclude MN occurrences
df = df[(df.preposition_1 != "MN") & (df.preposition_2 == "")]
df.head(2)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,tense,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,motion_type,preposition_1,preposition_2,preposition_3,preposition_4,comments
2,212418,BW>[,MT,Isaiah,1,23,JBW>,FRJK SWRRJM W XBRJ GNBJM KLW >HB CXD W RDP CLM...,,qal,impf,W RJB >LMNH L> JBW> >LJHM,RJB >LMNH,>LJHM,>L,to them,0,prep,prps,anim,det,prsf,simple,fictive,>L,,,,
3,1895234,BW>[,1Qisaa,Isaiah,1,23,JBW>,FRJKJ SWRRJM W XBRJ GNBJM KWLM >WHBJ CWXD RWDP...,000000 000000 0 0000 00000 00000 00000 0000 00...,qal,impf,W RJB >LMNH LW> JBW> >LJHM,RJB >LMNH,>LJHM,>L,to them,0,prep,prps,anim,det,prsf,simple,fictive,>L,,,,


In [8]:
df[(df.chapter == 52) & (df.verse_num == 2)]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,tense,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,motion_type,preposition_1,preposition_2,preposition_3,preposition_4,comments


In [5]:
len(set(df.lex))

25

In [6]:
set(df.lex)

{'<BR[',
 '<LH[',
 '<WP[',
 '<WZ[',
 'BW>[',
 'CQQ[',
 'CWB[',
 'CWR=[',
 'GLH[',
 'HLK[',
 'JRD[',
 'JY>[',
 'NGC[',
 'NHR[',
 'NPL[',
 'NWS[',
 'NWX[',
 'PF<[',
 'PNH[',
 'QRB[',
 'RWY[',
 'SBB[',
 'T<H[',
 'XLP[',
 'XSH['}

In [49]:
len(df[df.scroll=="MT"])

175

In [50]:
# COMPLEMENT: OCCURRENCE COUNT

cmpl_dic = {}

for i, row in df.iterrows():
    cmpl_info = (row.lex, row.book, row.chapter, row.verse_num)
    cmpl_dic.setdefault(row.scroll, {}).setdefault(cmpl_info, []).append(row.cmpl_constr)

stats = {}
for scroll, cmpls in cmpl_dic.items():
    for cmpl_info, cmpl_constrs in cmpls.items():
        for other_scroll, other_cmpls in cmpl_dic.items():
            if scroll >= other_scroll:
                continue
            other_cmpl_constr = other_cmpls.get(cmpl_info)
            if other_cmpl_constr is None:
                continue
            same = 0
            different = 0
            for c1, c2 in zip(cmpl_constrs, other_cmpl_constr):
                if c1 == c2:
                    same += 1
                else:
                    different += 1
                    #print(scroll, other_scroll, cmpl_info, c1, c2)
            stat = stats.get((scroll, other_scroll), [0, 0, 0])
            stats[(scroll, other_scroll)] = (stat[0] + same, stat[1] + different, stat[2] + same + different)
            
stats

{('1Qisaa', 'MT'): (165, 3, 168),
 ('1Q8', 'MT'): (65, 0, 65),
 ('1Q8', '1Qisaa'): (62, 1, 63)}

In [51]:
#df[(df.chapter == 52) & (df.verse_num == 11)]

In [52]:
# COMPLEMENT: OCCURRENCE COUNT AND PERCENTAGES, TABLE

pd.DataFrame([{
        "scroll1": scrolls[0], 
        "scroll2": scrolls[1], 
        "total_occ_scroll1": len(df[df.scroll == scrolls[0]]),
        "total_occ_scroll2": len(df[df.scroll == scrolls[1]]),
        "total_com_occ": stat[2],
        "total_agreement": stat[0],
        "total_disagreement": stat[1],
        "agreement_percentage": stat[0] / stat[2] * 100,
    }
    for scrolls, stat in stats.items()
])

,scroll1,scroll2,total_occ_scroll1,total_occ_scroll2,total_com_occ,total_agreement,total_disagreement,agreement_percentage
0,1Qisaa,MT,174,175,168,165,3,98.214286
1,1Q8,MT,65,175,65,65,0,100.000000
2,1Q8,1Qisaa,65,174,63,62,1,98.412698


In [53]:
# COMPLEMENT: DISAGREEMENT CASES
cmpl_dic = {}

# Constructing the dictionary from the dataframe
for i, row in df.iterrows():
    cmpl_info = (row.lex, row.book, row.chapter, row.verse_num)
    cmpl_dic.setdefault(row.scroll, {}).setdefault(cmpl_info, []).append(row.cmpl_constr)

results = []

# Analyzing and comparing complements
for scroll, cmpls in cmpl_dic.items():
    for cmpl_info, cmpl_constrs in cmpls.items():
        for other_scroll, other_cmpls in cmpl_dic.items():
            if scroll >= other_scroll:
                continue
            other_cmpl_constr = other_cmpls.get(cmpl_info)
            if other_cmpl_constr is None:
                continue
            
            for c1, c2 in zip(cmpl_constrs, other_cmpl_constr):
                if c1 != c2:  # Only storing results when they are different
                    result = {
                        'scroll': scroll,
                        'other_scroll': other_scroll,
                        'lex': cmpl_info[0],
                        'book': cmpl_info[1],
                        'chapter': cmpl_info[2],
                        'verse_num': cmpl_info[3],
                        'complement_1': c1,
                        'complement_2': c2
                    }
                    results.append(result)

# Create DataFrame
df_results = pd.DataFrame(results)

# If you want to see the DataFrame printed out
df_results
#stats

,scroll,other_scroll,lex,book,chapter,verse_num,complement_1,complement_2
0,1Qisaa,MT,BW>[,Isaiah,10,28,prep + dir-he,prep
1,1Qisaa,MT,<BR[,Isaiah,10,29,prep,vc
2,1Qisaa,MT,<LH[,Isaiah,60,7,prep,vc
3,1Q8,1Qisaa,<LH[,Isaiah,60,7,vc,prep


In [45]:
df[(df.chapter == 60) & (df.verse_num == 7) & (df.lex == "<LH[")]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,tense,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,motion_type,preposition_1,preposition_2,preposition_3,preposition_4,comments
521,232728,<LH[,MT,Isaiah,60,7,J<LW,KL Y>N QDR JQBYW LK >JLJ NBJWT JCRTWNK J<LW <L...,,qal,impf,J<LW <L RYWN MZBXJ,,MZBXJ,MZBX/,on my altar,0,vc,obj,inanim,det,subs,complex,factive fictive,,,,,
522,1916569,<LH[,1Qisaa,Isaiah,60,7,J<LW,KWL YW>N QDR JQBYW LK >JLJ NB>WT JCRTWNK W J<L...,000 0000 000 00000 000 0000 00000 000000000 0 ...,hif,impf,W J<LW L RYWN <L MZBXJ,,<L MZBXJ,<L MZBX/,on my altar,0,prep,obj,inanim,det,subs,complex,factive fictive,<L,,,,
523,1925919,<LH[,1Q8,Isaiah,60,7,J<LW,KL Y>N QDR JQBYW LK >JLJ NBJWT JCRTWNK J<LW RY...,00 000 000 00000 000 0000 00000 000000000 0000...,hif,impf,J<LW RYWN MZBXJ,,MZBXJ,MZBX/,on my altar,0,vc,obj,inanim,det,subs,complex,factive fictive,,,,,


In [25]:
# PREPOSITIONS: COUNT

prep_dic = {}

for i, row in df[df.cmpl_constr == "prep"].iterrows():
    prep_info = (row.lex, row.book, row.chapter, row.verse_num)
    preps = []
    for i in range(1, 5):
        prep = row[f"preposition_{i}"].strip()
        if prep:
            preps.append(prep)
    prep_dic.setdefault(row.scroll, {}).setdefault(prep_info, []).append(preps)

stats = {}
for scroll, scroll_preps in prep_dic.items():
    for prep_info, preps in scroll_preps.items():
        for other_scroll, other_scroll_preps in prep_dic.items():
            if scroll >= other_scroll:
                continue
            other_preps = other_scroll_preps.get(prep_info)
            if other_preps is None:
                continue
            same = 0
            different = 0
            for c1, c2 in zip(preps, other_preps):
                if c1 == c2:
                    same += 1
                else:
                    different += 1
                    #print(scroll, other_scroll, prep_info, c1, c2)
            stat = stats.get((scroll, other_scroll), [0, 0, 0])
            stats[(scroll, other_scroll)] = (stat[0] + same, stat[1] + different, stat[2] + same + different)

stats

{('1Qisaa', 'MT'): (127, 7, 134),
 ('1Q8', 'MT'): (49, 1, 50),
 ('1Q8', '1Qisaa'): (46, 3, 49)}

In [22]:
# PREPOSITIONS. DISAGREEMENT CASES BETWEEN THE THREE MANUSCRIPTS
prep_dic = {}

# Constructing the dictionary from the dataframe where 'cmpl_constr' is 'prep'
for i, row in df[df.cmpl_constr == "prep"].iterrows():
    prep_info = (row.lex, row.book, row.chapter, row.verse_num)
    preps = []
    for j in range(1, 5):
        prep = row[f"preposition_{j}"].strip()
        if prep:
            preps.append(prep)
    prep_dic.setdefault(row.scroll, {}).setdefault(prep_info, []).append(preps)

results = []

# Analyzing and comparing prepositions
for scroll, scroll_preps in prep_dic.items():
    for prep_info, preps in scroll_preps.items():
        for other_scroll, other_scroll_preps in prep_dic.items():
            if scroll >= other_scroll:
                continue
            other_preps = other_scroll_preps.get(prep_info)
            if other_preps is None:
                continue
            
            for c1, c2 in zip(preps, other_preps):
                if c1 != c2:  # Only storing results when they are different
                    result = {
                        'scroll': scroll,
                        'other_scroll': other_scroll,
                        'lex': prep_info[0],
                        'book': prep_info[1],
                        'chapter': prep_info[2],
                        'verse_num': prep_info[3],
                        'preposition_scroll_1': ", ".join(c1),
                        'preposition_scroll_2': ", ".join(c2)
                    }
                    results.append(result)

# Create DataFrame
df_results = pd.DataFrame(results)

# If you want to see the DataFrame printed out
df_results

,scroll,other_scroll,lex,book,chapter,verse_num,preposition_scroll_1,preposition_scroll_2
0,1Qisaa,MT,NHR[,Isaiah,2,2,<L,>L
1,1Qisaa,MT,BW>[,Isaiah,16,12,<L,>L
2,1Qisaa,MT,HLK[,Isaiah,18,2,L,>L
3,1Qisaa,MT,CWB[,Isaiah,37,7,L,>L
4,1Qisaa,MT,GLH[,Isaiah,53,1,>L,<L
5,1Qisaa,MT,BW>[,Isaiah,59,20,>L,L
6,1Qisaa,MT,BW>[,Isaiah,66,20,>L,<L
7,1Q8,1Qisaa,BW>[,Isaiah,16,12,>L,<L
8,1Q8,1Qisaa,CWB[,Isaiah,37,7,>L,L
9,1Q8,MT,GLH[,Isaiah,53,1,>L,<L


In [23]:
#print(df[(df.chapter == 54) & (df.verse_num == 9)])

In [26]:
# PREPOSITIONS: MANUSCRIPT AGREEMENT

def total_occ(scroll):
    scroll_preps = prep_dic[scroll]
    total = 0
    for preps in scroll_preps.values():
        total += len(preps)
    return total

pd.DataFrame([{
        "scroll1": scrolls[0], 
        "scroll2": scrolls[1], 
        "total_occ_scroll1": total_occ(scrolls[0]),
        "total_occ_scroll2": total_occ(scrolls[1]),
        "total_com_occ": stat[2],
        "total_agreement": stat[0],
        "total_disagreement": stat[1],
        "agreement_percentage": stat[0] / stat[2] * 100,
    }
    for scrolls, stat in stats.items()
])

,scroll1,scroll2,total_occ_scroll1,total_occ_scroll2,total_com_occ,total_agreement,total_disagreement,agreement_percentage
0,1Qisaa,MT,142,139,134,127,7,94.776119
1,1Q8,MT,50,139,50,49,1,98.000000
2,1Q8,1Qisaa,50,142,49,46,3,93.877551


In [37]:
len(df[(df.scroll == "1Qisaa") & (df.cmpl_constr == "prep + dir-he")])

2

In [40]:
df[(df.chapter == 10) & (df.verse_num == 28)]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,tense,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,motion_type,preposition_1,preposition_2,preposition_3,preposition_4,comments
78,216009,BW>[,MT,Isaiah,10,28,B>,B> <L <JT <BR B MGRWN L MKMF JPQJD KLJW,,qal,perf,B> <L <JT,,<L <JT,<L <JT/,to Aiath,0,prep,topo,inanim,det,nmpr,simple,factive,<L,,,,
79,216012,<BR[,MT,Isaiah,10,28,<BR,B> <L <JT <BR B MGRWN L MKMF JPQJD KLJW,,qal,perf,<BR B MGRWN,,B MGRWN,B MGRWN/,through Migron,0,prep,topo,inanim,det,nmpr,simple,factive,B,,,,
80,1898948,BW>[,1Qisaa,Isaiah,10,28,B>,B> <L <JTH <BR B MGRWN L MKMC JPQWD KLJW,00 00 00000 000 0 00000 0 0000 00000 00000 00,qal,perf,B> <L <JTH,,<L <JTH,<L <JT/,to Aiath,1,prep + dir-he,topo,inanim,det,nmpr,simple,factive,<L,,,,
81,1898951,<BR[,1Qisaa,Isaiah,10,28,<BR,B> <L <JTH <BR B MGRWN L MKMC JPQWD KLJW,00 00 00000 000 0 00000 0 0000 00000 00000 00,qal,perf,<BR B MGRWN,,B MGRWN,B MGRWN/,through Migron,0,prep,topo,inanim,det,nmpr,simple,factive,B,,,,


In [39]:
df[df.cmpl_constr == "prep + dir-he"]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,tense,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,motion_type,preposition_1,preposition_2,preposition_3,preposition_4,comments
61,215112,PNH[,MT,Isaiah,8,21,PNH,W <BR BH NQCH W R<B W HJH KJ JR<B W HTQYP W QL...,,qal,perf,W PNH L M<LH,,L M<LH,L M<L/,upwards,1,prep + dir-he,dir,inanim,und,subs,simple,factive,L,,,,
63,1898000,PNH[,1Qisaa,Isaiah,8,21,PNH,W <BR BH W NQCH W R<B W HJH KJ> JR<B JTQYP W Q...,0 000 000 0 0000 0 000 0 000 000 0000 00000 0 ...,qal,perf,W PNH L M<LH,,L M<LH,L M<L/,upwards,1,prep + dir-he,dir,inanim,und,subs,simple,factive,L,,,,
80,1898948,BW>[,1Qisaa,Isaiah,10,28,B>,B> <L <JTH <BR B MGRWN L MKMC JPQWD KLJW,00 00 00000 000 0 00000 0 0000 00000 00000 00,qal,perf,B> <L <JTH,,<L <JTH,<L <JT/,to Aiath,1,prep + dir-he,topo,inanim,det,nmpr,simple,factive,<L,,,,


In [38]:
set(df.scroll)

{'1Q8', '1Qisaa', 'MT'}

In [27]:
# PREPOSITION MISSING

stats = {}
for scroll, scroll_preps in prep_dic.items():
    for prep_info, preps in scroll_preps.items():
        for other_scroll, other_scroll_preps in prep_dic.items():
            if scroll == other_scroll:
                continue
            other_preps = other_scroll_preps.get(prep_info)
            if other_preps is None:
                continue
            non_missing = 0
            missing = 0
            for c1, c2 in zip_longest(preps, other_preps):
                if c2 is None:
                    missing += 1
                    print(scroll, other_scroll, prep_info, c1, c2)
                else:
                    non_missing += 1
            stat = stats.get((scroll, other_scroll), [0, 0, 0])
            stats[(scroll, other_scroll)] = (stat[0] + non_missing, stat[1] + missing, stat[2] + missing + non_missing)
            
stats

1Qisaa MT ('BW>[', 'Isaiah', 16, 12) ['>L'] None
1Qisaa 1Q8 ('BW>[', 'Isaiah', 16, 12) ['>L'] None


{('MT', '1Qisaa'): (135, 0, 135),
 ('MT', '1Q8'): (50, 0, 50),
 ('1Qisaa', 'MT'): (134, 1, 135),
 ('1Qisaa', '1Q8'): (49, 1, 50),
 ('1Q8', 'MT'): (50, 0, 50),
 ('1Q8', '1Qisaa'): (50, 0, 50)}

In [28]:
# PREPOSITION MISSING 

pd.DataFrame([{
        "scroll1": scrolls[0], 
        "scroll2": scrolls[1], 
        "total_occ_scroll1": total_occ(scrolls[0]),
        "total_occ_scroll2": total_occ(scrolls[1]),
        "total_com_occ": stat[2],
        "total_non_missing": stat[0],
        "total_missing_from_scroll2": stat[1],
        "agreement_percentage": stat[0] / stat[2] * 100,
    }
    for scrolls, stat in stats.items()
])

,scroll1,scroll2,total_occ_scroll1,total_occ_scroll2,total_com_occ,total_non_missing,total_missing_from_scroll2,agreement_percentage
0,MT,1Qisaa,139,142,135,135,0,100.000000
1,MT,1Q8,139,50,50,50,0,100.000000
2,1Qisaa,MT,142,139,135,134,1,99.259259
3,1Qisaa,1Q8,142,50,50,49,1,98.000000
4,1Q8,MT,50,139,50,50,0,100.000000
5,1Q8,1Qisaa,50,142,50,50,0,100.000000
